In [3]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

import sympy
from sympy import init_printing
init_printing()

# class quatum bbs

In [4]:
class quatum_Periodic_bbs:# p is the configuration of balls
    def __init__(self, l, q):
        ## パスを二進数表現
        self.path_amp =  np.zeros(2**l)
        self.q = q
        self.carrier = []
        self.energy = 0
        self.time = 0
        self.log = []
    
    def path2binary(self,p):##omitting 0 aligned the left side of the path.
        return int(str(p),2)
    
    def binary2path(self,n):
        return(int(bin(n)[2:]))
    
    def show_state():
        print("t = {}".format(self.time))
        print("amplitude is {}".format(self.path_amp))
    
    def set_state(amplitude_dist):
        self.path_amp =  amplitude_dist
        
    
    def P_operator(self, m): #m is the number of 1-color balls in the carrier. k is the positon.
        q = self.q
        return m, q**m #carrier, amplitude
    
    def Q_operator(self, m): #m is the number of 1-color balls in the carrier. k is the positon.
        return m + 1, 1

    def R_operator(self, m): #m is the number of 1-color balls in the carrier. k is the positon.
        q = self.q
        if m > 0:
            return m - 1, 1 - q**(2*m)
        elif m == 0:
            return 0, 1
    
        
        vl = np.sort(vl)
        if b == 1:
            if max(vl) == 1:
                vl = vl
                b_dash = b
            
            elif max(vl) == 2:
                vl[-1] = 1
                vl = np.sort(vl)
                b_dash = 2
            else:
                print("error")
        
        elif b == 2:
            if min(vl) == 2:
                vl = vl
                b_dash = b
            elif min(vl) == 1:
                vl[0] = 2
                vl = np.sort(vl)
                b_dash = 1
            else:
                print("error")
            
        return vl, b_dash
                
        
    
    def map_R(self, vl):
        p_dash = self.box.copy()
        for i in range(len(self.box)):
            b = self.box[i]
            vl, b_dash =  self.combitional_R(vl ,b)
            p_dash[i] = b_dash
        #self.box = p_dash
        return vl, p_dash
    
    def time_evolve(self, l, calc_energy = False, print_result = True):
        self.time += 1
        vl = np.ones(l)
        vl_new = np.zeros(l)
        p_temp = self.box.copy()
        #from IPython.core.debugger import Pdb; Pdb().set_trace()
        for i in range(l+100):
            vl = np.sort(vl)
            self.box = p_temp.copy()
            vl_new, p_dash = self.map_R(vl)
            if (vl != vl_new).any():
                vl[0] = 2
            elif (vl == vl_new).all():
                break
            else:
                print("error")
        if calc_energy == True:
            self.energy = np.count_nonzero((p_dash - p_temp) == -1)
        self.box = p_dash
        self.log.append(self.box)
        if print_result == True:
            if calc_energy == True:
                print("t = {0} | system = {1} | energy = {2}".format(self.time, self.box, self.energy))
            else:
                print("t = {0} | {1}".format(self.time, self.box))
        
        return vl_new, self.box
    
        
    


In [5]:
p = 11010110

In [6]:
np.array(list(str(p)),dtype = int)

array([1, 1, 0, 1, 0, 1, 1, 0])

## calculate amplitude

In [7]:
def P(m, q, z):
    return m, q**m*z**m

def Q(m, q, z):
    return m+1, z**m

def R(m, q, z):
    return m-1, (1 - q**(2*m))*z**m


def calc_amplitude(p1,p2, simple = True): #p1,p2 is int or str or np.array
    if type(p1) == int:
        p1 = np.array(list(str(p1)),dtype = int)
    if type(p2) == int:
        p2 = np.array(list(str(p2)),dtype = int)

    if type(p1) == str:
        p1 = np.array(list(p1),dtype = int)
    if type(p2) == str:
        p2 = np.array(list(p2),dtype = int)        

        if len(p1) != len(p2):
            print("the length between p1 and p2 is not same")
            return 0
    
    m = sympy.Symbol("m")
    q = sympy.Symbol("q")
    z = sympy.Symbol("z")
    ans = 1
    for i in range(len(p1)):
        if p1[i] == 0:
            if p2[i] == 0:
                m, ans_temp = P(m,q,z)
            elif p2[i] == 1:
                m, ans_temp = R(m,q,z)
            else:
                print("error")
                break
        elif p1[i] == 1:
            if p2[i] == 0:
                m, ans_temp = Q(m,q,z)
            elif p2[i] == 1:
                m, ans_temp_temp = P(m,q,z)
                ans_temp = ans_temp_temp * (-q)
            else:
                print("error")
                break
        else:
            print("error")
            break
        
        ans *= ans_temp
    answer = sympy.summation(ans, (m, 0, sympy.oo))
    answer = sympy.simplify(answer)
    return answer        
            

In [18]:
calc_amplitude("10000","00001")

⎧             3  4 ⎛     2⎞                                            
⎪            q ⋅z ⋅⎝1 - q ⎠                   │ 3  5│       │ 5  5│    
⎪      ──────────────────────────         for │q ⋅z │ < 1 ∧ │q ⋅z │ < 1
⎪       8  10    5  5    3  5                                          
⎪      q ⋅z   - q ⋅z  - q ⋅z  + 1                                      
⎪                                                                      
⎪        ⎛              ∞            ⎞                                 
⎪        ⎜             ___           ⎟                                 
⎪        ⎜     2       ╲             ⎟                                 
⎪   3  4 ⎜    q         ╲    3⋅m  5⋅m⎟               │ 5  5│           
⎪  q ⋅z ⋅⎜───────── +   ╱   q   ⋅z   ⎟           for │q ⋅z │ < 1       
⎪        ⎜ 5  5        ╱             ⎟                                 
⎪        ⎜q ⋅z  - 1    ‾‾‾           ⎟                                 
⎪        ⎝            m = 0          ⎠                          

In [20]:
ans1 = calc_amplitude("1100","1100")
ans2 = calc_amplitude("1100","1010")
ans3 = calc_amplitude("1100","1001")
ans4 = calc_amplitude("1100","0110")
ans5 = calc_amplitude("1100","0101")
ans6 = calc_amplitude("1100","0011")


In [22]:
x = sympy.simplify(ans1 + ans2 + ans3 + ans4 + ans5 + ans6)

In [23]:
sympy.factor(x)

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                   

In [24]:
ans1**2,ans2**2,ans3,ans4

⎛   ⎛⎧       1             │ 4  4│    ⎞         ⎛⎧       1             │ 4  4│
⎜   ⎜⎪  ───────────    for │q ⋅z │ < 1⎟         ⎜⎪  ───────────    for │q ⋅z │
⎜   ⎜⎪     4  4                       ⎟         ⎜⎪     4  4                   
⎜   ⎜⎪  - q ⋅z  + 1                   ⎟         ⎜⎪  - q ⋅z  + 1               
⎜   ⎜⎪                                ⎟         ⎜⎪                            
⎜   ⎜⎪  ∞                             ⎟     2   ⎜⎪  ∞                         
⎜-q⋅⎜⎨ ___                            ⎟, - q ⋅z⋅⎜⎨ ___                        
⎜   ⎜⎪ ╲                              ⎟         ⎜⎪ ╲                          
⎜   ⎜⎪  ╲    4⋅m  4⋅m                 ⎟         ⎜⎪  ╲    4⋅m  4⋅m             
⎜   ⎜⎪  ╱   q   ⋅z        otherwise   ⎟         ⎜⎪  ╱   q   ⋅z        otherwis
⎜   ⎜⎪ ╱                              ⎟         ⎜⎪ ╱                          
⎜   ⎜⎪ ‾‾‾                            ⎟         ⎜⎪ ‾‾‾                        
⎝   ⎝⎩m = 0                           ⎠         ⎝⎩m 

In [41]:
path = "./data/result.tex"

import datetime
def writeamp2result(filepath,path1,path2):
    answer = calc_amplitude(path1, path2)
    dt_now = datetime.datetime.now()
    with open(path, mode = "a") as f:
        f.write("-"*50)
        f.write("\\\\")
        f.write("\n")
        f.write(str(dt_now))
        f.write("\\\\")
        f.write("\n")
        f.write("path1 = ")
        f.write(str(path1))
        f.write("\\\\")
        f.write("\n")
        f.write("path2 = ")
        f.write(str(path2))
        f.write("\\\\")
        f.write("\n")
        f.write("$$")
        f.write("amplitude = ")
        f.write(sympy.latex(answer))
        f.write("$$")
        f.write("\n")
        f.write("-"*50)
        f.write("\\\\")
        f.write("\n")


In [45]:
aaa = ["10","01"]
for p2 in aaa:
    writeamp2result(path, "10", p2)

In [48]:
int(3/2)

In [ ]:
"""
def makeconflist(n,k):
    list
    
"""

In [51]:
list1100 = ["1100","1010","1001","0110","0101","0011"]
list1000 = ["1000","0100","0010","0001"]
list0000 = ["0000"]
list1110 = ["1110","1101","1011","0111"]
list1111 = ["1111"]

listn4 = []
listn4.append(list0000)
listn4.append(list1000)
listn4.append(list1100)
listn4.append(list1110)
listn4.append(list1111)

In [53]:
for k in range(len(listn4)):
    for p1 in listn4[k]:
        for p2 in listn4[k]:
            writeamp2result(path,p1,p2)
            print("Ampliutde" + str(p1) + "to" + str(p2) + "was written.")

Ampliutde0000to0000was written.
Ampliutde1000to1000was written.
Ampliutde1000to0100was written.
Ampliutde1000to0010was written.
Ampliutde1000to0001was written.
Ampliutde0100to1000was written.
Ampliutde0100to0100was written.
Ampliutde0100to0010was written.
Ampliutde0100to0001was written.
Ampliutde0010to1000was written.
Ampliutde0010to0100was written.
Ampliutde0010to0010was written.
Ampliutde0010to0001was written.
Ampliutde0001to1000was written.
Ampliutde0001to0100was written.
Ampliutde0001to0010was written.
Ampliutde0001to0001was written.
Ampliutde1100to1100was written.
Ampliutde1100to1010was written.
Ampliutde1100to1001was written.
Ampliutde1100to0110was written.
Ampliutde1100to0101was written.
Ampliutde1100to0011was written.
Ampliutde1010to1100was written.
Ampliutde1010to1010was written.
Ampliutde1010to1001was written.
Ampliutde1010to0110was written.
Ampliutde1010to0101was written.
Ampliutde1010to0011was written.
Ampliutde1001to1100was written.
Ampliutde1001to1010was written.
Ampliutd